In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import GaussianNB


# Load the dataset
df = pd.read_csv("Data/Election data.csv")
#Drop column F_INTFREQ
df.drop('F_INTFREQ', axis=1, inplace=True)

/var/folders/wp/79200sx97gq3ybg9qqyv3h140000gn/T/ipykernel_23883/2437674508.py:11: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/Election data.csv")


In [2]:
#XCITIZEN
#REPCONGEX
#DEMCONGEX

In [3]:
# Filter out rows where F_VOTEGEN2020 is not 'Joe Biden, the Democrat' or 'Donald Trump, the Republican'
df_filtered = df[(df['F_VOTEGEN2020'] == 'Joe Biden, the Democrat') | (df['F_VOTEGEN2020'] == 'Donald Trump, the Republican')]
# Replace string values with integers
df_filtered['F_VOTEGEN2020'] = df_filtered['F_VOTEGEN2020'].map({'Joe Biden, the Democrat': 1, 'Donald Trump, the Republican': 0})

# Reset the index after dropping rows
df_filtered.reset_index(drop=True, inplace=True)

df = df_filtered

print(df['F_VOTEGEN2020'].unique())

[1 0]


/var/folders/wp/79200sx97gq3ybg9qqyv3h140000gn/T/ipykernel_23883/963057324.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['F_VOTEGEN2020'] = df_filtered['F_VOTEGEN2020'].map({'Joe Biden, the Democrat': 1, 'Donald Trump, the Republican': 0})


In [4]:
# Mapping categorical values to numerical values
df['CANDHOUSAT_W116'] = df['CANDHOUSAT_W116'].map({
    'Refused': 0,
    'Not at all satisfied': 1,
    'Not too satisfied': 2,
    'Somewhat satisfied': 3,
    'Very satisfied': 4
})

# Resetting the index after dropping rows
df.reset_index(drop=True, inplace=True)

# Converting the column to float type
df['CANDHOUSAT_W116'] = df['CANDHOUSAT_W116'].astype(float)

# Printing unique values and info
print(df['CANDHOUSAT_W116'].unique())
print(df['CANDHOUSAT_W116'].info())



[2. 3. 1. 4. 0.]
<class 'pandas.core.series.Series'>
RangeIndex: 3859 entries, 0 to 3858
Series name: CANDHOUSAT_W116
Non-Null Count  Dtype  
--------------  -----  
3859 non-null   float64
dtypes: float64(1)
memory usage: 30.3 KB
None


In [5]:
# Replace 'Response out of range' with NaN in specified columns
df['THERMBIDEN_W116'].replace('Response out of range', np.nan, inplace=True)
df['THERMTRUMP_W116'].replace('Response out of range', np.nan, inplace=True)

# Convert the columns to float type
df['THERMBIDEN_W116'] = df['THERMBIDEN_W116'].astype(float)
df['THERMTRUMP_W116'] = df['THERMTRUMP_W116'].astype(float)

# Drop rows with NaN values from the specified columns
df.dropna(subset=['THERMBIDEN_W116', 'THERMTRUMP_W116'], inplace=True)

# print(df[['THERMBIDEN_W116', 'THERMTRUMP_W116']].apply(pd.Series.unique))
print(df['THERMTRUMP_W116'].info())


<class 'pandas.core.series.Series'>
Index: 3831 entries, 0 to 3858
Series name: THERMTRUMP_W116
Non-Null Count  Dtype  
--------------  -----  
3831 non-null   float64
dtypes: float64(1)
memory usage: 59.9 KB
None


In [6]:
#Drop all NA values
df.dropna(subset=['INSTN_MIL_W116'], inplace=True)


df['INSTN_MIL_W116'] = df['INSTN_MIL_W116'].map({
    'Refused': 0,
    'Negative effect': 1,
    'Positive effect': 2
})

df.reset_index(drop=True, inplace=True)

df['INSTN_MIL_W116'] = df['INSTN_MIL_W116'].astype(float)

print(df['INSTN_MIL_W116'].unique())
print(df['INSTN_MIL_W116'].info())

[2. 1. 0.]
<class 'pandas.core.series.Series'>
RangeIndex: 1916 entries, 0 to 1915
Series name: INSTN_MIL_W116
Non-Null Count  Dtype  
--------------  -----  
1916 non-null   float64
dtypes: float64(1)
memory usage: 15.1 KB
None


In [10]:
# Define features (X) and target variable (Y)
vectorizer = CountVectorizer()
# X = df[['political_affiliation', 'THERMBIDEN_W116', 'THERMTRUMP_W116', 'satisfaction_with_direction', 'CANDHOUSAT_W116', 'government_regulation_opinion', 'INSTN_MIL_W116', 'assistance_programs_opinion']]
X = df[['THERMBIDEN_W116', 'THERMTRUMP_W116','CANDHOUSAT_W116', 'INSTN_MIL_W116']]
Y = df['F_VOTEGEN2020']

# Model Training
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
nb_model = MultinomialNB()

nb_model.fit(X_train, y_train)
# Model Evaluation
train_score = nb_model.score(X_train, y_train)
test_score = nb_model.score(X_test, y_test)
print("Train Score:", train_score)
print("Test Score:", test_score)


Train Score: 0.9373368146214099
Test Score: 0.9192708333333334


In [12]:
# Assuming you have your new data stored in a DataFrame called 'new_data'
new_X = df[['THERMBIDEN_W116', 'THERMTRUMP_W116', 'CANDHOUSAT_W116', 'INSTN_MIL_W116']]

# Make predictions
predictions = nb_model.predict(new_X)

# Now 'predictions' contains the predicted outcomes
print(predictions)


[1 0 0 ... 1 1 1]


In [16]:
from collections import Counter

# Assuming 'predictions' is the output of your model predictions
prediction_counts = Counter(predictions)

# Get the count of zeros and ones
count_zeros = prediction_counts[0]
count_ones = prediction_counts[1]

print("Count of zeros:", count_zeros)
print("Count of ones:", count_ones)

if count_zeros > count_ones:
    print("Trump wins")
elif count_ones > count_zeros:
    print("Biden wins")



Count of zeros: 834
Count of ones: 1082
Biden wins
